In [6]:

# set working directory
import os
os.chdir("/Users/antonwiklund/Documents/Understanding-Tool-Aware-AI-Agents/code/tool-aware-ai-testing")

from src.database.connection import get_connection
from tabulate import tabulate
import pandas as pd


In [7]:
conn, cur = get_connection()

In [8]:
cur.execute("""
    SELECT 
        r.id,
        p.prompt,
        t.model_name,
        r.tool_calls,
        r.time_taken,
        r.success_rate,
        r.error_type,
        r.created_at
    FROM results r
    JOIN prompts p ON r.prompt_id = p.id
    JOIN test_runs t ON r.test_run_id = t.id
    ORDER BY r.created_at DESC;
""")
results = cur.fetchall()


In [9]:
# Convert to pandas DataFrame with column names
df = pd.DataFrame(results, columns=[
    'ID', 'Prompt', 'Model', 'Tool Calls', 
    'Time Taken', 'Success', 'Error', 'Created At'
])

In [10]:
# Display the results as a formatted table
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))


+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+-----------+---------+------------

In [11]:
# Calculate and display statistics
print("\n=== Test Run Statistics ===")

# Success rate by model
model_stats = df.groupby('Model').agg({
    'Success': ['count', 'mean'],
    'Time Taken': ['mean', 'std']
}).round(3)

print("\nSuccess Rate by Model:")
print(tabulate(model_stats, headers=[
    'Total Tests', 'Success Rate', 'Avg Time', 'Std Time'
], tablefmt='psql'))

# Success rate by individual run
# First, we need to add test_run_id to our DataFrame
cur.execute("""
    SELECT 
        r.id,
        r.test_run_id,
        p.prompt,
        t.model_name,
        r.tool_calls,
        r.time_taken,
        r.success_rate,
        r.error_type,
        r.created_at,
        p.correct_tools
    FROM results r
    JOIN prompts p ON r.prompt_id = p.id
    JOIN test_runs t ON r.test_run_id = t.id
    ORDER BY r.test_run_id DESC, r.created_at DESC;
""")
results = cur.fetchall()

# Update DataFrame with test_run_id
df = pd.DataFrame(results, columns=[
    'ID', 'Test Run ID', 'Prompt', 'Model', 'Tool Calls', 
    'Time Taken', 'Success', 'Error', 'Created At', 'Correct Tools'
])

# Group by Test Run ID
run_stats = df.groupby('Test Run ID').agg({
    'Success': ['count', 'mean'],
    'Time Taken': ['mean', 'std'],
    'Model': 'first',  # Get the model name for this test run
    'Created At': ['min', 'max'],  # Get start and end time
    'Tool Calls': lambda x: len(set([item for sublist in x for item in sublist]))  # unique tools used
}).round(3)

# Rename columns for better readability
run_stats.columns = [
    'Total Tests', 'Success Rate', 'Avg Time', 'Time Std Dev', 
    'Model', 'Start Time', 'End Time', 'Unique Tools'
]

print("\nSuccess Rate by Test Run:")
print(tabulate(run_stats, headers='keys', tablefmt='psql'))

tool_counts = {}
for tools in df['Tool Calls']:
    for tool in tools:
        tool_counts[tool] = tool_counts.get(tool, 0) + 1

# Overall statistics
overall_stats = pd.DataFrame({
    'Metric': [
        'Total Test Runs',
        'Total Tests',
        'Overall Success Rate',
        'Average Time per Test',
        'Total Unique Tools Used',
        'Most Used Tool'
    ],
    'Value': [
        len(run_stats),
        len(df),
        f"{df['Success'].mean()*100:.1f}%",
        f"{df['Time Taken'].mean():.3f} seconds",
        len(set([tool for tools in df['Tool Calls'] for tool in tools])),
        max(tool_counts.items(), key=lambda x: x[1])[0]
    ]
})

print("\nOverall Statistics:")
print(tabulate(overall_stats, headers='keys', tablefmt='psql', showindex=False))

# Tool usage statistics
print("\nTool Usage Statistics:")
tool_counts = {}
for tools in df['Tool Calls']:
    for tool in tools:
        tool_counts[tool] = tool_counts.get(tool, 0) + 1

tool_stats = pd.DataFrame.from_dict(
    tool_counts, 
    orient='index', 
    columns=['Count']
).sort_values('Count', ascending=False)

# Add percentage column
total_calls = tool_stats['Count'].sum()
tool_stats['Percentage'] = (tool_stats['Count'] / total_calls * 100).round(1)
tool_stats['Percentage'] = tool_stats['Percentage'].apply(lambda x: f"{x}%")

print(tabulate(tool_stats, headers=['Tool', 'Usage Count', 'Usage %'], tablefmt='psql'))


=== Test Run Statistics ===

Success Rate by Model:
+-------------+---------------+----------------+------------+------------+
|             |   Total Tests |   Success Rate |   Avg Time |   Std Time |
|-------------+---------------+----------------+------------+------------|
| gpt-4o-mini |           929 |          0.822 |      0.004 |      0.029 |
+-------------+---------------+----------------+------------+------------+

Success Rate by Test Run:
+---------------+---------------+----------------+------------+----------------+-------------+----------------------------+----------------------------+----------------+
|   Test Run ID |   Total Tests |   Success Rate |   Avg Time |   Time Std Dev | Model       | Start Time                 | End Time                   |   Unique Tools |
|---------------+---------------+----------------+------------+----------------+-------------+----------------------------+----------------------------+----------------|
|             1 |           929 |  

In [12]:
# Analyze tool failures
print("\nTool Failure Analysis:")

# First, get the correct tools for each prompt
cur.execute("""
    SELECT 
        r.id,
        r.tool_calls,
        p.correct_tools
    FROM results r
    JOIN prompts p ON r.prompt_id = p.id
    ORDER BY r.created_at DESC;
""")
tool_comparison_results = cur.fetchall()

# Create a dictionary to track failures for each tool
tool_failures = {}
total_tool_expectations = {}

# Analyze each result
for result in tool_comparison_results:
    _, used_tools, expected_tools = result
    
    # Convert array strings to Python lists if they're not already
    if isinstance(expected_tools, str):
        expected_tools = expected_tools.strip('{}').split(',')
    if isinstance(used_tools, str):
        used_tools = used_tools.strip('{}').split(',')
    
    # Convert to sets for comparison
    expected_tools_set = set(tool.strip() for tool in expected_tools)
    used_tools_set = set(tool.strip() for tool in used_tools)
    
    # Track total expectations for each tool
    for tool in expected_tools_set:
        tool = tool.strip()  # Remove any whitespace
        total_tool_expectations[tool] = total_tool_expectations.get(tool, 0) + 1
        
        # If tool was expected but not used, count as failure
        if tool not in used_tools_set:
            tool_failures[tool] = tool_failures.get(tool, 0) + 1

# Create failure statistics DataFrame
if total_tool_expectations:  # Only create stats if we have data
    failure_stats = pd.DataFrame({
        'Tool': list(total_tool_expectations.keys()),
        'Total Expected Uses': list(total_tool_expectations.values()),
        'Failed Uses': [tool_failures.get(tool, 0) for tool in total_tool_expectations.keys()]
    })

    # Calculate failure rate
    failure_stats['Failure Rate'] = (failure_stats['Failed Uses'] / failure_stats['Total Expected Uses'] * 100).round(1)
    failure_stats['Failure Rate'] = failure_stats['Failure Rate'].apply(lambda x: f"{x}%")

    # Sort by number of failures in descending order
    failure_stats = failure_stats.sort_values('Failed Uses', ascending=False)

    print(tabulate(failure_stats, headers=['Tool', 'Expected Uses', 'Failed Uses', 'Failure Rate'], 
                  tablefmt='psql', showindex=False))
else:
    print("No tool usage data available for analysis.")


Tool Failure Analysis:
+---------------------------+-----------------+---------------+----------------+
| Tool                      |   Expected Uses |   Failed Uses | Failure Rate   |
|---------------------------+-----------------+---------------+----------------|
| statistical_analysis_tool |             191 |            55 | 28.8%          |
| code_tool                 |             105 |            47 | 44.8%          |
| database_tool             |             176 |            24 | 13.6%          |
| summary_tool              |             169 |            23 | 13.6%          |
| calendar_tool             |             130 |            15 | 11.5%          |
| task_management_tool      |             148 |             8 | 5.4%           |
| search_web_tool           |             113 |             2 | 1.8%           |
+---------------------------+-----------------+---------------+----------------+


In [14]:
# Analyze failed prompts
print("\nFailed Prompts Analysis:")

# Query to get failed prompts with details
cur.execute("""
    SELECT 
        p.prompt,
        p.correct_tools,
        r.tool_calls,
        r.error_type,
        r.time_taken,
        t.model_name
    FROM results r
    JOIN prompts p ON r.prompt_id = p.id
    JOIN test_runs t ON r.test_run_id = t.id
    WHERE r.success_rate = false
    ORDER BY r.created_at DESC;
""")
failed_results = cur.fetchall()

# Create DataFrame for failed prompts
failed_df = pd.DataFrame(failed_results, columns=[
    'Prompt', 'Expected Tools', 'Used Tools', 
    'Error Type', 'Time Taken', 'Model'
])

if len(failed_df) > 0:
    # Add percentage of total prompts that failed
    failure_rate = (len(failed_df) / len(df) * 100)
    print(f"\nTotal Failed Prompts: {len(failed_df)} ({failure_rate}% of all prompts)")
    
    # Group by error type if present
    if failed_df['Error Type'].notna().any():
        print("\nFailures by Error Type:")
        error_stats = failed_df['Error Type'].value_counts()
        print(tabulate(
            error_stats.reset_index(), 
            headers=['Error Type', 'Count'], 
            tablefmt='psql'
        ))
    
    # Display failed prompts with details
    print("\nDetailed Failed Prompts:")
    # Format the output for better readability
    failed_df['Time Taken'] = failed_df['Time Taken'].round(3)
    failed_df['Expected Tools'] = failed_df['Expected Tools'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    failed_df['Used Tools'] = failed_df['Used Tools'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    
    print(tabulate(failed_df, headers='keys', tablefmt='psql', showindex=False))
else:
    print("No failed prompts found in the results.")


Failed Prompts Analysis:

Total Failed Prompts: 165 (17.761033369214207% of all prompts)

Detailed Failed Prompts:
+----------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------+------------------------------------------------------------------------------------------------------+--------------+--------------+-------------+
| Prompt                                                                                                                                       | Expected Tools                                  | Used Tools                                                                                           | Error Type   |   Time Taken | Model       |
|----------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------+------